In [ ]:
def getNeighborMedian(nbr):
    if nbr == 'MeadowV':
        return 88000 
    elif nbr == 'IDOTRR':
        return 103000
    elif nbr == 'BrDale':
        return 106000  
    elif nbr == 'OldTown':
        return 119000   
    elif nbr == 'Edwards':
        return 121750
    elif nbr == 'BrkSide':
        return 124300
    elif nbr == 'Sawyer':
        return 135000
    elif nbr == 'Blueste':
        return 137500
    elif nbr == 'SWISU':
        return 139500
    elif nbr == 'NAmes':
        return 140000
    elif nbr == 'NPkVill':
        return 146000
    elif nbr == 'Mitchel':
        return 153500
    elif nbr == 'SawyerW':
        return 179900    
    elif nbr == 'Gilbert':
        return 181000   
    elif nbr == 'NWAmes':
        return 182900    
    elif nbr == 'Blmngtn':
        return 191000   
    elif nbr == 'CollgCr':
        return 197200
    elif nbr == 'ClearCr':
        return 200250
    elif nbr == 'Crawfor':
        return 200624
    elif nbr == 'Veenker':
        return 218000
    elif nbr == 'Somerst':
        return 225500
    elif nbr == 'Timber':
        return 228475
    elif nbr == 'StoneBr':
        return 278000  
    elif nbr == 'NoRidge':
        return 301500   
    elif nbr == 'NridgHt':
        return 315000  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
#panda is a read/write tool

train_file = "c:\\house\\train.csv"
test_file = "c:\\house\\test.csv"

# comma delimited is the default
hsData = pd.read_csv(train_file, header = 0)


hsData['House_Freshment'] = hsData['YearBuilt'] + hsData['YearRemodAdd']
hsData['House_Freshment'].apply(lambda year: year - 3700)

hsData['FireplaceQu'] = hsData['FireplaceQu'].fillna('NA')

hsData['Neighborhood_Median'] = hsData['Neighborhood'].apply(getNeighborMedian)

hsData['Total_FlrSF'] = hsData['1stFlrSF'] + hsData['2ndFlrSF']
#hsData = hsData[hsData["Total_FlrSF"] < 4000]
#hsData = hsData[hsData["Total_FlrSF"] > 500]
#hsData = hsData[hsData["GrLivArea"] < 2500]
#hsData = hsData[hsData["Overall_QQ"] < 100]
#hsData = hsData[hsData["Overall_QQ"] > 30]

hsData['Porch_SF'] = hsData['OpenPorchSF'] + hsData['EnclosedPorch'] + hsData['3SsnPorch'] + hsData['ScreenPorch']


hsData['MSSubClass'] = hsData['MSSubClass'].apply(str)
hsData['MS_Zone_Class'] = hsData['MSZoning'] + hsData['MSSubClass']
msc_median = hsData.pivot_table(index="MS_Zone_Class", values="SalePrice", aggfunc=np.median)

for i, item in enumerate(msc_median.index):
    hsData.loc[hsData['MS_Zone_Class'] == msc_median.index[i],'MS_Zone_Class'] = msc_median.values[i]

hsData['MS_Zone_Class'] = pd.to_numeric(hsData['MS_Zone_Class'], errors='coerce')    

hsData['Exter_QQ'] = hsData['ExterQual'] + hsData['ExterCond']


hsData['Air_OQ'] = hsData['CentralAir']*10 + hsData['HeatingQC']

hsData['Exterior_Mix'] = hsData['Exterior1st'] + hsData['Exterior2nd']
'''
ex_bar = hsData.pivot_table(index="Exterior_Mix", values="SalePrice", aggfunc=np.median)

for i, item in enumerate(ex_bar.index):
    hsData.loc[hsData['Exterior_Mix'] == ex_bar.index[i],'Exterior_Mix'] = ex_bar.values[i]
    
hsData['Exterior_Mix'] = pd.to_numeric(hsData['Exterior_Mix'], errors='coerce')    
'''

hsData['Conditions'] = hsData['Condition1'] + hsData['Condition2']

hsData['BsmtQual'] = hsData['BsmtQual'].fillna('NA')
hsData['BsmtCond'] = hsData['BsmtCond'].fillna('NA')
hsData['Bsmt_QQ'] = hsData['BsmtQual'] + hsData['BsmtCond']


#0.27 before
hsData['GarageQual'] = hsData['GarageQual'].fillna('NA')
hsData['GarageCond'] = hsData['GarageCond'].fillna('NA')
hsData['Garage_QQ'] = hsData['GarageQual'] + hsData['GarageCond']

hsData.info()
#hsData.describe()


In [ ]:
numeric_features = hsData.select_dtypes(include=[np.number])
numeric_features.dtypes

corr = numeric_features.corr()

print (corr['SalePrice'].sort_values(ascending=False)[:30], '\n')
print (corr['SalePrice'].sort_values(ascending=False)[-30:])

In [ ]:

null_data = pd.DataFrame(hsData.isnull().sum().sort_values(ascending=False)[:25])
null_data.columns = ["Null Count"]
null_data.index.name = "Feature"
null_data

In [ ]:
tmp_hsData = hsData.copy()

msc_bar = hsData.pivot_table(index="MS_Zone_Class", values="SalePrice", aggfunc=np.median)

msc_bar_sorted = msc_bar.sort_values(by="SalePrice")
msc_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("general zoning classification")
plt.ylabel("Sale price")
plt.show()

#print msc_bar_sorted

for i, item in enumerate(msc_bar_sorted.index):
    tmp_hsData.loc[tmp_hsData['MS_Zone_Class'] == msc_bar_sorted.index[i],'MS_Zone_Class'] = msc_bar_sorted.values[i]
    
tmp_hsData['MS_Zone_Class'] = pd.to_numeric(tmp_hsData['MS_Zone_Class'], errors='coerce')    
#print tmp_hsData['MS_Zone_Class']


In [ ]:
#Not useful, too many zeros
hsData_vnr_trim = hsData[hsData["MasVnrArea"] < 10000]
#hsData_vnr_trim = hsData_vnr_trim[hsData_flr_trim["MasVnrArea"] > 500]

plt.scatter(x=hsData_vnr_trim["MasVnrArea"], y=np.log(hsData_vnr_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Masonry veneer area square feet")
plt.show()

In [ ]:
#Very useful
hsData_flr_trim = hsData[hsData["Total_FlrSF"] < 3500]
hsData_flr_trim = hsData_flr_trim[hsData_flr_trim["Total_FlrSF"] > 500]

plt.scatter(x=hsData_flr_trim["Total_FlrSF"], y=np.log(hsData_flr_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Floor square feet")
plt.show()

In [ ]:
rf_bar = hsData.pivot_table(index="RoofMatl", values="SalePrice", aggfunc=np.median)

rf_bar_sorted = rf_bar.sort_values(by="SalePrice")
rf_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Roof material")
plt.ylabel("sale price")
plt.show()

print rf_bar_sorted

#hsData['RoofMatl'].replace(['Roll', 'ClyTile', 'CompShg', 'Tar&Grv', 'Metal', 'Membran', 'WdShake', 'WdShngl'], [137000,160000,162000,167000,180000,241500,242000,332500], inplace=True)

In [ ]:
msc_bar = hsData.pivot_table(index="MSSubClass", values="SalePrice", aggfunc=np.median)

msc_bar_sorted = msc_bar.sort_values(by="SalePrice")
msc_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("MSSubClass")
plt.ylabel("Sale price")
plt.show()

print msc_bar_sorted

#hsData['MSSubClass'].replace(['180', '30', '45', '190', '50', '90', '85', '40', '160', '70', '20', '75', '80', '120', '60'], [88500,99900,107500,128250,132000,135980,140750,142500,146000,156000,159250,163500,166500,192000,215200], inplace=True)

In [ ]:
mz_bar = hsData.pivot_table(index="MSZoning", values="SalePrice", aggfunc=np.median)

mz_bar_sorted = mz_bar.sort_values(by="SalePrice")
mz_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("MSZoning")
plt.ylabel("Sale price")
plt.show()

print mz_bar_sorted

#hsData['MSZoning'].replace(['C (all)','RM','RH','RL','FV'], [74700,120500,136500,174000,205950], inplace=True)

In [ ]:
lc_bar = hsData.pivot_table(index="LotConfig", values="SalePrice", aggfunc=np.median)

lc_bar_sorted = lc_bar.sort_values(by="SalePrice")
lc_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("LotConfig")
plt.ylabel("Sale price")
plt.show()

print lc_bar_sorted

In [ ]:
ls_bar = hsData.pivot_table(index="LandSlope", values="SalePrice", aggfunc=np.median)

ls_bar_sorted = ls_bar.sort_values(by="SalePrice")
ls_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("LandSlope")
plt.ylabel("Sale price")
plt.show()

print ls_bar_sorted

In [ ]:
tmp_hsData = hsData.copy()

ex_bar = hsData.pivot_table(index="Exterior_Mix", values="SalePrice", aggfunc=np.median)

ex_bar_sorted = ex_bar.sort_values(by="SalePrice")
ex_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Exterior covering on house")
plt.ylabel("Sale price")
plt.show()

#print ex_bar_sorted

for i, item in enumerate(ex_bar.index):
    tmp_hsData.loc[tmp_hsData['Exterior_Mix'] == ex_bar.index[i],'Exterior_Mix'] = ex_bar.values[i]
    
tmp_hsData['Exterior_Mix'] = pd.to_numeric(tmp_hsData['Exterior_Mix'], errors='coerce')    
print tmp_hsData['MS_Zone_Class']

In [ ]:
ex1_bar = hsData.pivot_table(index="Exterior1st", values="SalePrice", aggfunc=np.median)

ex1_bar_sorted = ex1_bar.sort_values(by="SalePrice")
ex1_bar_sorted.plot(kind="bar", color="blue")


plt.xlabel("Exterior covering on house")
plt.ylabel("Sale price")
plt.show()

print ex1_bar_sorted

#hsData['Exterior1st'].replace(['BrkComm', 'AsphShn', 'CBlock', 'AsbShng', 'WdShing', 'Wd Sdng', 'MetalSd', 'Stucco', 'HdBoard', 'BrkFace', 'Plywood', 'VinylSd', 'CemntBd', 'Stone', 'ImStucc'], \
#[71000,100000,105000,108000,128700,138943,139000,144000,149900,165750,167450,200000,236500,258500,262000], inplace=True)


In [ ]:
ex2_bar = hsData.pivot_table(index="Exterior2nd", values="SalePrice", aggfunc=np.median)

ex2_bar_sorted = ex2_bar.sort_values(by="SalePrice")
ex2_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Exterior covering on house")
plt.ylabel("Sale price")
plt.show()

print ex2_bar_sorted

#hsData['Exterior2nd'].replace(['CBlock', 'AsbShng', 'Wd Sdng', 'Wd Shng', 'MetalSd', 'AsphShn', 'Stucco', 'Brk Cmn', 'HdBoard', 'BrkFace', 'Plywood', 'Stone', 'ImStucc', 'VinylSd', 'CmentBd', 'Other'], \
#[105000,111000,138000,138225,138750,139000,142000,147000,155000,160000,160750,177000,187600,200070,238750,319000], inplace=True)


In [ ]:
#Useful
nb_bar = hsData.pivot_table(index="Neighborhood", values="SalePrice", aggfunc=np.median)

nb_bar_sorted = nb_bar.sort_values(by="SalePrice")
nb_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("neighborhood")
plt.ylabel("sale price")
plt.show()

print nb_bar_sorted


In [ ]:
st_bar = hsData.pivot_table(index="SaleType", values="SalePrice", aggfunc=np.median)

st_bar_sorted = st_bar.sort_values(by="SalePrice")
st_bar_sorted.plot(kind="bar", color="red")

plt.xlabel("SaleType")
plt.ylabel("sale price")
plt.show()

print st_bar_sorted

#hsData['SaleType'].replace(['Oth', 'ConLI', 'COD', 'ConLD', 'ConLw', 'WD', 'CWD', 'New', 'Con'], [116050,125000,139000,140000,144000,158000,188750,247453,269600], inplace=True)

In [ ]:
sc_bar = hsData.pivot_table(index="SaleCondition", values="SalePrice", aggfunc=np.median)

sc_bar_sorted = sc_bar.sort_values(by="SalePrice")
sc_bar_sorted.plot(kind="bar", color="red")

plt.xlabel("SaleCondition")
plt.ylabel("sale price")
plt.show()

print sc_bar_sorted

#hsData['SaleCondition'].replace(['AdjLand', 'Abnorml', 'Family', 'Alloca', 'Normal', 'Partial'], [104000,130000,140500,148145,160000,244600], inplace=True)

In [ ]:
ele_bar = hsData.pivot_table(index="Electrical", values="SalePrice", aggfunc=np.median)

ele_bar_sorted = ele_bar.sort_values(by="SalePrice")
ele_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Electrical system")
plt.ylabel("sale price")
plt.show()

print ele_bar_sorted

#hsData['Electrical'].replace(['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr'], [67000,82000,115000,121250,170000], inplace=True)

In [ ]:
bsm_bar = hsData.pivot_table(index="Bsmt_QQ", values="SalePrice", aggfunc=np.median)

bsm_bar_sorted = bsm_bar.sort_values(by="SalePrice")
bsm_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("Bsmt Qual+Cond")
plt.ylabel("sale price")
plt.show()

print bsm_bar_sorted

'''
hsData['Bsmt_QQ'].replace(['FaPo', 'NANA', 'FaTA', 'FaFa', 'TAFa', 'TATA', 'TAGd', 'GdFa', \
                            'GdTA', 'GdGd', 'ExGd', 'ExTA'], \
                             [64,112, 114,114,114,135,164,182,192,199,275,318], inplace=True)
'''

In [ ]:
#useful
hs_bar = hsData.pivot_table(index="KitchenQual", values="SalePrice", aggfunc=np.median)

hs_bar_sorted = hs_bar.sort_values(by="SalePrice")
hs_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("HouseStyle")
plt.ylabel("sale price")
plt.show()

print hs_bar_sorted

#hsData['KitchenQual'].replace(['NA', Fa', 'TA', 'Gd', 'Ex'], [105565,115000,137000,201400,316750], inplace=True)

In [ ]:
gg_bar = hsData.pivot_table(index="Garage_QQ", values="SalePrice", aggfunc=np.median)

gg_bar_sorted = gg_bar.sort_values(by="SalePrice")
gg_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("Garage Qual+Cond")
plt.ylabel("sale price")
plt.show()

print gg_bar_sorted

'''
hsData['Garage_QQ'].replace(['PoPo', 'NANA', 'FaFa', 'FaPo', 'FaTA', 'ExEx', 'TAFa', 'TAGd', \
                             'TATA', 'GdTA', 'GdGd', 'ExTA'], \
                             [97,100,110,116,116,125,125,144,171,209,227,475], inplace=True)
'''

In [ ]:
ext_bar = hsData.pivot_table(index="Exter_QQ", values="SalePrice", aggfunc=np.median)

ext_bar_sorted = ext_bar.sort_values(by="SalePrice")
ext_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("Exterior Qual+Cond")
plt.ylabel("sale price")
plt.show()

print ext_bar_sorted

'''
hsData['Exter_QQ'].replace(['FaFa', 'TAPo', 'FaTA', 'TAFa', 'TAEx', 'TATA', 'TAGd', 'GdGd', \
                             'GdTA', 'ExGd', 'ExEx', 'ExTA'], \
                             [61,77,99,119,140,140,140,210,221,267,325,374], inplace=True)
'''

In [ ]:
# Living area, orignal
#hsData_lv_trim = hsData
hsData_lv_trim = hsData[hsData["GrLivArea"] < 3500]
hsData_lv_trim = hsData_lv_trim[hsData_lv_trim["GrLivArea"] > 500]
plt.scatter(x=hsData_lv_trim["GrLivArea"], y=hsData_lv_trim["SalePrice"])
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Above grade (ground) living area square feet")
plt.show()

In [ ]:
# Living area, log
hsData_lv_trim= hsData[hsData["GrLivArea"] < 3500]
hsData_lv_trim= hsData_lv_trim[hsData_lv_trim["GrLivArea"] > 500]
plt.scatter(x=np.log(hsData_lv_trim["GrLivArea"]), y=np.log(hsData_lv_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Above grade (ground) living area square feet")
plt.show()

In [ ]:
# Garage area

hsData_gar_trim = hsData[hsData["GarageArea"] < 1200]

plt.scatter(x=hsData_gar_trim["GarageArea"], y=np.log(hsData_gar_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Sale Price")
plt.xlabel("Size of garage in square feet")
plt.show()

In [ ]:
hsData_gar_trim = hsData[hsData["GarageArea"] < 1200]

plt.scatter(x=np.log(hsData_gar_trim["GarageArea"]), y=np.log(hsData_gar_trim["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Size of garage in square feet")
plt.show()

#hsData_trim.info()

In [ ]:
hsData_year_trim = hsData.copy()

plt.scatter(x=hsData_year_trim['YearBuilt'], y=np.log(hsData_year_trim['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()

hsData_year_trim.info()

In [ ]:
hsData_year_shift = hsData.copy()
hsData_year_shift['YearBuilt'] = hsData_year_shift['YearBuilt'].apply(lambda x: x - 1850)
#hsData_year_shift = hsData_year_shift[hsData_year_shift["YearBuilt"] > 40]

plt.scatter(x=hsData_year_shift['YearBuilt'], y=np.log(hsData_year_shift['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()

In [ ]:
hsData_year_shift = hsData.copy()
hsData_year_shift['YearBuilt'] = hsData_year_shift['YearBuilt'].apply(lambda x: 2030 - x)
hsData_year_shift = hsData_year_shift[hsData_year_shift["YearBuilt"] < 140]

plt.scatter(x=hsData_year_shift['YearBuilt'], y=np.log(hsData_year_shift['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()

In [ ]:
hsData_year_shift = hsData['House_Freshment']

plt.scatter(x=hsData_year_shift, y=np.log(hsData['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()


In [ ]:

plt.scatter(x=hsData['MiscVal'], y=np.log(hsData['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()

In [ ]:
hsData_month = hsData.copy()
#hsData_year_shift['Year'] = hsData_year_shift['YearBuilt'].apply(lambda x: 2030 - x)
#hsData_year_shift = hsData_year_shift[hsData_year_shift["YearBuilt"] < 140]

plt.scatter(x=hsData_month['MoSold'], y=np.log(hsData_month['SalePrice']))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Original construction date")
plt.show()

In [ ]:
hsData = hsData[hsData["Total_FlrSF"] < 3500]
hsData = hsData[hsData["Total_FlrSF"] > 500]
plt.scatter(x=hsData["Total_FlrSF"], y=np.log(hsData["SalePrice"]))
plt.grid(True)
plt.ylabel("Ln of Sale Price")
plt.xlabel("Floor square feet")
plt.show()

In [ ]:
#Noise
cd1_bar = hsData.pivot_table(index="Condition1", values="SalePrice", aggfunc=np.median)

cd1_bar_sorted = cd1_bar.sort_values(by="SalePrice")
cd1_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("condition 1")
plt.ylabel("sale price")
plt.show()

print cd1_bar_sorted

#hsData['Condition1'].replace(['Artery', 'Feedr', 'RRAe', 'Norm', 'RRAn', 'RRNe', 'PosN', 'PosA', 'RRNn'], [119550,140000,142500,166500,171495,190750,200000,212500,214000], inplace=True)

In [ ]:
#Noise
cd2_bar = hsData.pivot_table(index="Condition2", values="SalePrice", aggfunc=np.median)

cd2_bar_sorted = cd2_bar.sort_values(by="SalePrice")
cd2_bar_sorted.plot(kind="bar", color="yellow")

plt.xlabel("condition 2")
plt.ylabel("sale price")
plt.show()

print cd2_bar_sorted

#hsData['Condition2'].replace(['RRNn', 'Artery', 'Feedr', 'RRAn', 'Norm', 'RRAe', 'PosN', 'PosA'], [96750,106500,127500,136905,163500,190000,284875,325000], inplace=True)

In [ ]:
#Noise
cds_bar = hsData.pivot_table(index="Conditions", values="SalePrice", aggfunc=np.median)

cds_bar_sorted = cds_bar.sort_values(by="SalePrice")
cds_bar_sorted.plot(kind="bar", color="green")

plt.xlabel("condition 1+2")
plt.ylabel("sale price")
plt.show()

print cds_bar_sorted

'''
hsData['Conditions'].replace(['FeedrFeedr', 'FeedrRRNn', 'ArteryArtery', 'ArteryNorm', \
                              'RRNnFeedr', 'RRAnFeedr', 'FeedrRRAn', 'FeedrNorm', \
                             'RRAeNorm', 'NormNorm', 'RRAnNorm', 'FeedrRRAe', \
                             'RRNeNorm', 'PosNNorm', 'PosANorm', 'RRNnNorm', \
                             'PosNPosN', 'ArteryPosA'], \
                             [85000,96750,106500,119900,128000,133500,136905,141000,142500,166500,178950,190000,190750,200000,212500,252000,284875,325000], inplace=True)
'''

In [ ]:
ca_bar = hsData.pivot_table(index="CentralAir", values="SalePrice", aggfunc=np.median)

ca_bar_sorted = ca_bar.sort_values(by="SalePrice")
ca_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("CentralAir")
plt.ylabel("sale price")
plt.show()

print ca_bar_sorted


In [ ]:
gt_bar = hsData.pivot_table(index="GarageType", values="SalePrice", aggfunc=np.median)

gt_bar_sorted = gt_bar.sort_values(by="SalePrice")
gt_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("GarageType")
plt.ylabel("sale price")
plt.show()

print gt_bar_sorted

#hsData['GarageType'].replace(['CarPort', 'Detchd', 'Basment', '2Types', 'Attchd', 'BuiltIn'], [108000,129500,148000,159000,185000,227500], inplace=True)

In [ ]:
fn_bar = hsData.pivot_table(index="Functional", values="SalePrice", aggfunc=np.median)

fn_bar_sorted = fn_bar.sort_values(by="SalePrice")
fn_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Functional")
plt.ylabel("sale price")
plt.show()

print fn_bar_sorted

#hsData['Functional'].replace(['Maj2', 'Sev', 'Mod', 'Min1', 'Min2', 'Maj1', 'Typ'], [85000,129000,137900,139000,140000,140750,165500], inplace=True)

In [ ]:
bld_bar = hsData.pivot_table(index="BldgType", values="SalePrice", aggfunc=np.median)

bld_bar_sorted = bld_bar.sort_values(by="SalePrice")
bld_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("BldgType")
plt.ylabel("sale price")
plt.show()

print bld_bar_sorted

#hsData['BldgType'].replace(['2fmCon','Duplex','Twnhs','1Fam','TwnhsE'], [127500,135980,137500,167900,172200], inplace=True)

In [ ]:
hs_bar = hsData.pivot_table(index="HouseStyle", values="SalePrice", aggfunc=np.median)

hs_bar_sorted = hs_bar.sort_values(by="SalePrice")
hs_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("HouseStyle")
plt.ylabel("sale price")
plt.show()

print hs_bar_sorted

#hsData['HouseStyle'].replace(['1.5Unf','1.5Fin','2.5Unf','SFoyer','1Story','SLvl','2Story','2.5Fin'], [111250,132000,133900,135960,154750,164500,190000,194000], inplace=True)

In [ ]:
hsData['Sold_Time'] = (hsData['YrSold'] - 2005)*12 + hsData['MoSold']

fn_bar = hsData.pivot_table(index="Sold_Time", values="SalePrice", aggfunc=np.median)

fn_bar_sorted = fn_bar.sort_values(by="SalePrice")
fn_bar_sorted.plot(kind="bar", color="blue")

plt.xlabel("Sold_Time")
plt.ylabel("sale price")
plt.show()

print fn_bar_sorted